In [2]:
import os
import sys
import urllib.request
import pandas as pd
import datetime
import pymysql
import re
import json

In [15]:
client_id = "YOUR_CLIENT_ID"
client_secret = "YOUR_CLIENT_SECRET"

start_date = '2016-01-01'
today = datetime.date.today().strftime('%Y-%m-%d')
url = "https://openapi.naver.com/v1/datalab/search";
body = "{\"startDate\":\""+start_date+"\",\"endDate\":\""+today+"\",\"timeUnit\":\"week\",\"keywordGroups\":[{\"groupName\":\"경제\",\"keywords\":[\"경제\"]}]}";

request = urllib.request.Request(url)
request.add_header("X-Naver-Client-Id",client_id)
request.add_header("X-Naver-Client-Secret",client_secret)
request.add_header("Content-Type","application/json")
response = urllib.request.urlopen(request, data=body.encode("utf-8"))
rescode = response.getcode()
if(rescode==200):
    response_body = response.read()
    scrapped = response_body.decode('utf-8')
else:
    print("Error Code:" + rescode)

In [16]:
jsonResult = json.loads(scrapped)
print(jsonResult['results'][0]['data'])

[{'period': '2015-12-28', 'ratio': 19.65163}, {'period': '2016-01-04', 'ratio': 26.05846}, {'period': '2016-01-11', 'ratio': 25.16599}, {'period': '2016-01-18', 'ratio': 27.38639}, {'period': '2016-01-25', 'ratio': 24.72622}, {'period': '2016-02-01', 'ratio': 23.01888}, {'period': '2016-02-08', 'ratio': 23.62248}, {'period': '2016-02-15', 'ratio': 25.2824}, {'period': '2016-02-22', 'ratio': 23.76476}, {'period': '2016-02-29', 'ratio': 28.85228}, {'period': '2016-03-07', 'ratio': 31.58144}, {'period': '2016-03-14', 'ratio': 28.44701}, {'period': '2016-03-21', 'ratio': 26.88626}, {'period': '2016-03-28', 'ratio': 26.26972}, {'period': '2016-04-04', 'ratio': 25.94205}, {'period': '2016-04-11', 'ratio': 25.57126}, {'period': '2016-04-18', 'ratio': 26.74829}, {'period': '2016-04-25', 'ratio': 25.11425}, {'period': '2016-05-02', 'ratio': 22.536}, {'period': '2016-05-09', 'ratio': 27.78304}, {'period': '2016-05-16', 'ratio': 27.96412}, {'period': '2016-05-23', 'ratio': 29.37397}, {'period': '

In [17]:
def extract_year_and_month(period):
    year = period.split('-')[0]
    month = period.split('-')[1]
    day = period.split('-')[2]
    return year, month, day

# DB 연동 및 table 생성

In [18]:
conn = pymysql.connect(host = "127.0.0.1", user = [USER], passwd = [PASSWORD], db = [DATABASE])
cur = conn.cursor()

In [19]:
"""
검색어 트랜드 정보 저장할 table 생성
"""

create_table_query = """
CREATE TABLE IF NOT EXISTS naver_counting(
    id BIGINT(7) NOT NULL AUTO_INCREMENT,
    year bigint(4) NOT NULL,
    month bigint(2) NOT NULL,
    day bigint(2) NOT NULL,
    cnt double,
    primary key(id) )
    charset=utf8mb4;
"""
cur.execute(create_table_query)

0

In [20]:
cur.execute("desc naver_counting")
table = cur.fetchall()
pd.DataFrame(list(table))

,0,1,2,3,4,5
0,id,bigint(7),NO,PRI,None,auto_increment
1,year,bigint(4),NO,,None,
2,month,bigint(2),NO,,None,
3,day,bigint(2),NO,,None,
4,cnt,double,YES,,None,


In [21]:
"""
db table에 저장
"""

for result in jsonResult['results'][0]['data']:
    query = """ 
    Insert into naver_counting (year, month, day, cnt) values (%d, %d, %d, %f) ;
    """
    year, month, day = extract_year_and_month(result["period"])
    naver_cnt = result["ratio"]

    mystring = (query % (int(year), int(month), int(day), float(naver_cnt)))
    print(mystring)
    cur.execute(mystring)

values (2016, 10, 10, 25.347070) ;
    
 
    Insert into naver_counting (year, month, day, cnt) values (2016, 10, 17, 25.480720) ;
    
 
    Insert into naver_counting (year, month, day, cnt) values (2016, 10, 24, 25.032330) ;
    
 
    Insert into naver_counting (year, month, day, cnt) values (2016, 10, 31, 26.235230) ;
    
 
    Insert into naver_counting (year, month, day, cnt) values (2016, 11, 7, 27.558850) ;
    
 
    Insert into naver_counting (year, month, day, cnt) values (2016, 11, 14, 25.278080) ;
    
 
    Insert into naver_counting (year, month, day, cnt) values (2016, 11, 21, 27.778730) ;
    
 
    Insert into naver_counting (year, month, day, cnt) values (2016, 11, 28, 27.265670) ;
    
 
    Insert into naver_counting (year, month, day, cnt) values (2016, 12, 5, 24.204530) ;
    
 
    Insert into naver_counting (year, month, day, cnt) values (2016, 12, 12, 25.368620) ;
    
 
    Insert into naver_counting (year, month, day, cnt) values (2016, 12, 19, 21.212380)

In [22]:
# 확인
query = """ 
select * from naver_counting ;
"""
cur.execute(query)
tmp=pd.DataFrame(list(cur.fetchall()))
tmp

,0,1,2,3,4
0,1,2015,12,28,19.65163
1,2,2016,1,4,26.05846
2,3,2016,1,11,25.16599
3,4,2016,1,18,27.38639
4,5,2016,1,25,24.72622
...,...,...,...,...,...
245,246,2020,9,7,39.85944
246,247,2020,9,14,36.24213
247,248,2020,9,21,36.60429
248,249,2020,9,28,25.29102


In [10]:
conn.commit()
cur.close()
conn.close()